In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold  
from sklearn.svm import LinearSVC
import warnings
warnings.filterwarnings("ignore")  
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.calibration import CalibratedClassifierCV



df = pd.read_csv('wdbc.csv', header = None)


output = df.iloc[:,0]


for i in np.arange(0,df.shape[0]):
    if output.iloc[i] == 'B':
        output.iloc[i] = 0
    else:
        output.iloc[i] = 1



features = df.iloc[:,1:]




features_normal = normalize(features)


features_normal_df = pd.DataFrame(features_normal)


norm_whole = pd.concat([features_normal_df,output],axis=1)


col_head = norm_whole.columns


norm_whole.columns= ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X11','X12','X13','X14','X15','X16','X17','X18','X19','X20','X21','X22','X23','X24','X25','X26','X27','X28','X29','X30','y']


norm_whole.sort_values(by=['y'],inplace=True)


norm_whole.reset_index(drop=True,inplace=True)


X_y_0 = norm_whole.iloc[0:357,:]
X_y_1 = norm_whole.iloc[357:569,:]
X_y_1.reset_index(drop=True,inplace=True)

C_selected = pd.DataFrame(index=np.arange(1,31),columns=['Selected Penalty'])



Precision_Train = pd.DataFrame(index=np.arange(1,31),columns=['Precision Train'])

Recall_Train = pd.DataFrame(index=np.arange(1,31),columns=['Recall Train'])

F1_Train = pd.DataFrame(index=np.arange(1,31),columns=['F1 Train'])

Accuracy_Train = pd.DataFrame(index=np.arange(1,31),columns=['Accuracy Train'])

AUC_Train = pd.DataFrame(index=np.arange(1,31),columns=['AUC Train'])

Train_Error = pd.DataFrame(index=np.arange(1,31),columns=['Train Error for best selected C'])





Precision_Test = pd.DataFrame(index=np.arange(1,31),columns=['Precision Test'])

Recall_Test = pd.DataFrame(index=np.arange(1,31),columns=['Recall Test'])

F1_Test = pd.DataFrame(index=np.arange(1,31),columns=['F1 Test'])

Accuracy_Test = pd.DataFrame(index=np.arange(1,31),columns=['Accuracy Test'])

AUC_Test = pd.DataFrame(index=np.arange(1,31),columns=['AUC Test'])

Test_Error = pd.DataFrame(index=np.arange(1,31),columns=['Test Error for best selected C'])

Test_Error_Estimated = pd.DataFrame(index=np.arange(1,31),columns=['Test Error Estimated, via 5 fold CV, for best C'])




# Note the overall measure containing dataframes have been capitalized, to ensure that variable conflict is avoided, as all
# local quantities are in small letters





for mc in np.arange(0,30):
    
    print(mc)   # to keep run track
    
    
    X_y_train_0,X_y_test_0 = train_test_split(X_y_0,test_size = 0.2,shuffle=True)
    X_y_train_1,X_y_test_1 = train_test_split(X_y_1,test_size = 0.2,shuffle=True)



    X_y_train = pd.concat([X_y_train_0,X_y_train_1],axis=0)
    X_y_test = pd.concat([X_y_test_0,X_y_test_1],axis=0)


    

    X_y_train = shuffle(X_y_train)
    X_y_test = shuffle(X_y_test)



    X_y_train.reset_index(drop=True,inplace=True)
    X_y_test.reset_index(drop=True,inplace=True)


    X_train = X_y_train.drop(columns=['y'])
    X_test = X_y_test.drop(columns=['y'])
    y_train = X_y_train['y'].astype(int)
    y_test =  X_y_test['y'].astype(int)
    


    
    kf = KFold(n_splits=5,shuffle=True)

    C = np.array([10**(-1),10**(0),10**(1),10**(2),10**(3),10**(4),10**(5),10**(6),10**(7)])  

    

    error_df = pd.DataFrame(index = C, columns = ['Estimated Test Errors'])

    for i in np.arange(0,C.size):

        cv_ite = -1
        cv_error_vector = np.zeros(5)   

        for train,test in kf.split(X_train):

            cv_ite = cv_ite + 1  
            X_train_cv,X_test_cv = X_train.iloc[train,:],X_train.iloc[test,:]
            y_train_cv,y_test_cv = y_train[train],y_train[test]
            clf = LinearSVC(penalty='l1',dual=False,C=C[i]).fit(X_train_cv,y_train_cv)
            y_test_cv_pred = clf.predict(X_test_cv)

            mis = 0
            for l in np.arange(0,X_test_cv.shape[0]):
                if y_test_cv_pred[l] != y_test_cv.iloc[l]:
                    mis = mis + 1

            percent_error = (mis/X_test_cv.shape[0])*100
            cv_error_vector[cv_ite] = percent_error


        error_df.iloc[i,0] = np.mean(cv_error_vector)


    


    min_error = np.min(error_df.to_numpy().flatten())
    min_C = C[np.argmin(error_df.to_numpy().flatten())]
    
    Test_Error_Estimated.iloc[mc,0] = min_error
    
    C_selected.iloc[mc,0] = min_C

    
    
    

    clf_final = LinearSVC(penalty='l1',dual=False,C=min_C).fit(X_train,y_train)
    y_test_pred = clf_final.predict(X_test)

    mis = 0
    for l in np.arange(0,X_test.shape[0]):
        if y_test_pred[l] != y_test.iloc[l]:
            mis = mis + 1

    test_error = (mis/X_test.shape[0])*100
    

    Test_Error.iloc[mc,0] = test_error
    
    
    y_train_pred = clf_final.predict(X_train)

    mis = 0
    for l in np.arange(0,X_train.shape[0]):
        if y_train_pred[l] != y_train.iloc[l]:
            mis = mis + 1

    train_error = (mis/X_train.shape[0])*100
    

    
    Train_Error.iloc[mc,0] = train_error
    

    confusion_matrix_train = confusion_matrix(y_train,y_train_pred)

    

    confusion_matrix_train_df = pd.DataFrame(confusion_matrix_train,index=['Actually 0','Actually 1'],columns=['Predicted 0','Predicted 1'])
    


    precision_train = (confusion_matrix_train[1][1]/(confusion_matrix_train[1][1]+confusion_matrix_train[0][1]))*100
    recall_train = (confusion_matrix_train[1][1]/(confusion_matrix_train[1][1]+confusion_matrix_train[1][0]))*100

    Precision_Train.iloc[mc,0] = precision_train
    Recall_Train.iloc[mc,0] = recall_train


    accuracy_train = ((confusion_matrix_train[0][0]+confusion_matrix_train[1][1])/(confusion_matrix_train[0][0]+confusion_matrix_train[1][1]+confusion_matrix_train[0][1]+confusion_matrix_train[1][0]))*100
    f1_train = ((2*(precision_train/100)*(recall_train/100))/((precision_train/100)+(recall_train/100))) # divided by 100 as precision and recall specified in percentage
    
    Accuracy_Train.iloc[mc,0] = accuracy_train
    F1_Train.iloc[mc,0] = f1_train

    

    confusion_matrix_test = confusion_matrix(y_test,y_test_pred)

    

    confusion_matrix_test_df = pd.DataFrame(confusion_matrix_test,index=['Actually 0','Actually 1'],columns=['Predicted 0','Predicted 1'])
    


    precision_test = (confusion_matrix_test[1][1]/(confusion_matrix_test[1][1]+confusion_matrix_test[0][1]))*100
    recall_test = (confusion_matrix_test[1][1]/(confusion_matrix_test[1][1]+confusion_matrix_test[1][0]))*100
    
    Precision_Test.iloc[mc,0] = precision_test
    Recall_Test.iloc[mc,0] = recall_test

    


    accuracy_test = ((confusion_matrix_test[0][0]+confusion_matrix_test[1][1])/(confusion_matrix_test[0][0]+confusion_matrix_test[1][1]+confusion_matrix_test[0][1]+confusion_matrix_test[1][0]))*100
    f1_test = ((2*(precision_test/100)*(recall_test/100))/((precision_test/100)+(recall_test/100))) # divided by 100 as precision and recall specified in percentage
    
    
    Accuracy_Test.iloc[mc,0] = accuracy_test
    F1_Test.iloc[mc,0] = f1_test
    

    clf_prob = LinearSVC(penalty='l1',dual=False,C=min_C)
    calibrated_svc = CalibratedClassifierCV(clf_prob,method='sigmoid') # probabilities estimated via sigmoid method
    calibrated_svc.fit(X_train,y_train)
    y_train_pred_prob = calibrated_svc.predict_proba(X_train)

    


    y_train_pred_prob_0 = y_train_pred_prob[:,0]
    

    thresh = np.arange(0,1.01,0.01)
    ROC_df = pd.DataFrame(index=thresh,columns=['FPR','TPR'])
    for i in np.arange(0,thresh.size):
        y_train_pred_roc = (y_train_pred_prob_0 <= thresh[i]).astype(int)
        confusion_matrix_train_roc = confusion_matrix(y_train,y_train_pred_roc)
        ROC_df.iloc[i,0] = confusion_matrix_train_roc[0][1]/(confusion_matrix_train_roc[0][1]+confusion_matrix_train_roc[0][0])
        ROC_df.iloc[i,1] = confusion_matrix_train_roc[1][1]/(confusion_matrix_train_roc[1][1]+confusion_matrix_train_roc[1][0])
    

    AUC_train = metrics.auc(ROC_df['FPR'],ROC_df['TPR'])
    AUC_Train.iloc[mc,0] = AUC_train      # Capital T distinguishes the vector from the local scalar
    

    y_test_pred_prob = calibrated_svc.predict_proba(X_test)
    y_test_pred_prob_0 = y_test_pred_prob[:,0]



    thresh = np.arange(0,1.01,0.01)
    ROC_df_test = pd.DataFrame(index=thresh,columns=['FPR','TPR'])
    for i in np.arange(0,thresh.size):
        y_test_pred_roc = (y_test_pred_prob_0 <= thresh[i]).astype(int)
        confusion_matrix_test_roc = confusion_matrix(y_test,y_test_pred_roc)
        ROC_df_test.iloc[i,0] = confusion_matrix_test_roc[0][1]/(confusion_matrix_test_roc[0][1]+confusion_matrix_test_roc[0][0])
        ROC_df_test.iloc[i,1] = confusion_matrix_test_roc[1][1]/(confusion_matrix_test_roc[1][1]+confusion_matrix_test_roc[1][0])
   


   


    AUC_test = metrics.auc(ROC_df_test['FPR'],ROC_df_test['TPR'])
    
    AUC_Test.iloc[mc,0] = AUC_test       # Capital T distinguishes the vector from the local scalar
    

    
print('The best C, penalty, selected for each Monte Carlo iterations is :\n',C_selected)
    
print('The Precision for Train for the Monte Carlo iterations (for best C) is : \n',Precision_Train)
print('The Recall for Train for the Monte Carlo iterations (for best C) is : \n',Recall_Train)
print('The Accuracy for Train for the Monte Carlo iterations (for best C) is : \n',Accuracy_Train)
print('The F1 for Train for the Monte Carlo iterations (for best C) is : \n',F1_Train)
print('The training error for the Monte Carlo iterations (for best C) is : \n',Train_Error)
print('The AUC for Train for the Monte Carlo iterations (for best C) is : \n',AUC_Train)



print('The Precision for Test for the Monte Carlo iterations (for best C) is : \n',Precision_Test)
print('The Recall for Test for the Monte Carlo iterations (for best C) is : \n',Recall_Test)
print('The Accuracy for Test for the Monte Carlo iterations (for best C) is : \n',Accuracy_Test)
print('The F1 for Test for the Monte Carlo iterations (for best C) is : \n',F1_Test)
print('The test error for the Monte Carlo iterations (for best C) is : \n',Test_Error)
print('The AUC for Test for the Monte Carlo iterations (for best C) is : \n',AUC_Test)


print('The test error estimated, for each Monte Carlo iteration, via 5-fold CV, for best/selected C (minimum CV test error for C range, in each iteration, to select corresponding C) is : \n',Test_Error_Estimated)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
The best C, penalty, selected for each Monte Carlo iterations is :
    Selected Penalty
1             1e+07
2             10000
3             10000
4             10000
5             10000
6            100000
7             10000
8             10000
9             10000
10            1e+06
11            1e+06
12            1e+06
13            10000
14            10000
15            1e+06
16            10000
17            1e+07
18            10000
19            10000
20           100000
21            10000
22           100000
23            1e+07
24            1e+07
25            10000
26            1e+06
27            10000
28            1e+07
29           100000
30           100000
The Precision for Train for the Monte Carlo iterations (for best C) is : 
    Precision Train
1          98.2353
2          98.7952
3          98.1928
4          98.7805
5          98.7805
6          98.1818
7          97.5904
8    

Quite a few results were needed, which have all been obtained above in appropriate dataframes

NOTE : Precision, Recall, Accuracy for Train and Test are in %
       Training Errors, Test Errors, Test Error Estimates are in %
       F1 score and AUC are in their normal units (per unit)(not in %)
       C is the best C selected, penalty selected for each Monte-Carlo iteration

Average of all these quantities over the 30 Monte-Carlo iterations :

Averaging will be done for all the quantities, except the penalty, C term
For the C-value, the overall C-value/penalty that minimizes the test-error/test-error estimate, will be considered as the penalty selected MOST FREQUENTLY in the 30 Monte-Carlo iterations

In [11]:
ovrl_precision_train = Precision_Train.mean(axis=0)
ovrl_recall_train = Recall_Train.mean(axis=0)
ovrl_accuracy_train = Accuracy_Train.mean(axis=0)
ovrl_F1_train = F1_Train.mean(axis=0)
ovrl_auc_train = AUC_Train.mean(axis=0)
ovrl_train_error = Train_Error.mean(axis=0)


ovrl_precision_test = Precision_Test.mean(axis=0)
ovrl_recall_test = Recall_Test.mean(axis=0)
ovrl_accuracy_test = Accuracy_Test.mean(axis=0)
ovrl_F1_test = F1_Test.mean(axis=0)
ovrl_auc_test = AUC_Test.mean(axis=0)
ovrl_test_error = Test_Error.mean(axis=0)


ovrl_test_error_estimate = Test_Error_Estimated.mean(axis=0)
ovrl_C_selected = C_selected.mode(axis=0)


print('FOR TRAIN \n')

print('\nThe overall train precision is : \n',pd.DataFrame(ovrl_precision_train).iloc[0,0],'%')
print('\nThe overall train recall is : \n',pd.DataFrame(ovrl_recall_train).iloc[0,0],'%')
print('\nThe overall train accuracy is : \n',pd.DataFrame(ovrl_accuracy_train).iloc[0,0],'%')
print('\nThe overall train F1 is : \n',pd.DataFrame(ovrl_F1_train).iloc[0,0])
print('\nThe overall train AUC is : \n',pd.DataFrame(ovrl_auc_train).iloc[0,0])
print('\nThe overall train error is : \n',pd.DataFrame(ovrl_train_error).iloc[0,0],'%')

print('\nFOR TEST \n')

print('\nThe overall test precision is : \n',pd.DataFrame(ovrl_precision_test).iloc[0,0],'%')
print('\nThe overall test recall is : \n',pd.DataFrame(ovrl_recall_test).iloc[0,0],'%')
print('\nThe overall test accuracy is : \n',pd.DataFrame(ovrl_accuracy_test).iloc[0,0],'%')
print('\nThe overall test F1 is : \n',pd.DataFrame(ovrl_F1_test).iloc[0,0])
print('\nThe overall test AUC is : \n',pd.DataFrame(ovrl_auc_test).iloc[0,0])
print('\nThe overall test error is : \n',pd.DataFrame(ovrl_test_error).iloc[0,0],'%')


print('\nThe overall test error estimated is : \n',pd.DataFrame(ovrl_test_error_estimate).iloc[0,0],'%')
print('\nThe overall value of penalty selected is : \n',pd.DataFrame(ovrl_C_selected).iloc[0,0])


print('\nNOTE : All this has been averaged over 30 Monte-Carlo Trials, with train and test selected randomly \n')
print('\nNOTE : For Penalty, instead of average, mode has been used as measure of central tendency over Monte Carlo iterations \n')
print('\nNOTE : All the above values are, obviously, averaged over the best selected C in each case (in each Monte Carlo iteration) \n')
print('\nNOTE : All values except C (penalty), F1 Score, AUC are in %')





FOR TRAIN 


The overall train precision is : 
 98.63938056207739 %

The overall train recall is : 
 97.14003944773177 %

The overall train accuracy is : 
 98.4361233480176 %

The overall train F1 is : 
 0.9788222449346705

The overall train AUC is : 
 0.9987193328488875

The overall train error is : 
 1.5638766519823792 %

FOR TEST 


The overall test precision is : 
 94.78215931441036 %

The overall test recall is : 
 95.73643410852713 %

The overall test accuracy is : 
 96.3768115942029 %

The overall test F1 is : 
 0.9518987796800954

The overall test AUC is : 
 0.9945951765719211

The overall test error is : 
 3.6231884057970998 %

The overall test error estimated is : 
 3.3404965404965408 %

The overall value of penalty selected is : 
 10000.0

NOTE : All this has been averaged over 30 Monte-Carlo Trials, with train and test selected randomly 


NOTE : For Penalty, instead of average, mode has been used as measure of central tendency over Monte Carlo iterations 


NOTE : All the 

The above shows the averaged summary over all Monte-Carlo iterations